### 库龄分析

库存周转率是在某特定的周期，销售成本与存货平均余额的比率。用以衡量一定时期内存货资产的周转速度，是反映企业的供应链的整体效率的绩效指标之一，而且很多企业都把它作为整体经营业绩的考核指标之一。

#### 1. 库存账龄分析的概念

提到库存账龄，就不得不说到库存周转率。库存周转率是在某特定的周期，销售成本与存货平均余额的比率。用以衡量一定时期内存货资产的周转速度，是反映企业的供应链的整体效率的绩效指标之一，而且很多企业都把它作为整体经营业绩的考核指标之一。而库存账龄是在某时间节点，某种或者某类存货的库存时间的加权平均值。很明显，库存周转率越高，库存账龄越低，但是二者又不是反比关系（比较简单的证明就是同样的平均库存，入库时间的不同就会引起库存账龄很大的差异），所以虽然这二者有着千丝万缕的联系，但是不能简单的把库存账龄看成库存周转率的一个衍生指标来对待。

#### 2. 库存账龄分析的目的

在库存账龄分析中,其目的主要有以下两点：

一、库存成本的控制。与应收账款的账龄一样，存货的库存账龄越长，说明周转越慢，占压的资金也就越多。这也就是我们大家平常所说的滞销品。对于滞销品，应该分析其原因，从计划的源头控制入手，才能最有效的降低无效的库存，达到降低库存总额的目的。滞销品实际上包括两部分：不周转的物料和周转慢的物料，对于不周转的物料，显然除了上述工作外，还应该做相应的处理：比如代用或者变卖。

二、对于超龄的库存，也应该做好存货损失的准备，更真实的反映库存的实际价值。

#### 3. 库存账龄的计算

手工计算库存账龄是很困难的。在ERP系统中，库存账龄的计算相当方便。如果要计算某一仓库或者全部存货的库存账龄（虽然该数字可能没有实际的意义），那么公式如下：

    库存账龄=∑(批次入库数量*批次入库时间/统计时点库存总额)。

下面我举例一说明：

在2008年10月6日，存货A的库存为1000，系统自动搜索入库单，得到如下入库单：

    入库单号 日期 数量
    NO.025 10月5日入库200；
    NO.023 10月4日入库300；
    NO.022 10月3日入库300；
    NO.020 10月2日入库400；
    NO.015 10月1日入库100。
    
系统默认先进先出的原则，从10月6日的入库单倒推满1000为止，也就是200（10月5日）+300（10月4日）+300（10月3日）+200（10月2日入库的400只取值200即可）=1000.
那么：`库存账龄 = 200/1000*1 + 300/1000*2 + 300/1000*3 + 200/1000*4 = 0.2 + 0.6 + 0.9 + 0.8 = 2.5天`

库存账龄还有另外一种报表方式，反映的是库存账龄的集中度，举例二如下：

库存30天以下的数量： 1000；
库存30天--90天的数量： 2000；
库存90天--180天的数量：1500；
库存180天以上的数量： 500。

https://baike.baidu.com/item/%E5%BA%93%E9%BE%84%E5%88%86%E6%9E%90

In [1]:
import pandas as pd
from datetime import datetime

我们清洗好了一份采购明细样例数据, 按采购日期降序排列

In [2]:
df = pd.read_csv('./data/stockage-sample-data.csv')
df.head()

,sku_id,当前库存,采购入库日期,采购入库数量
0,1007271270102,1296,2021-02-22,1296
1,1008903694810,1388,2021-02-22,1388
2,1006671011405,1296,2021-02-22,1296
3,1008229644607,240,2021-02-22,240
4,1006379474960,180,2021-02-22,180


我们先计算出采购日期距离今天有多少天

In [3]:
current_datetime = datetime.now()
print(current_datetime)

2021-03-02 14:12:40.513775


In [4]:
df['days'] = (current_datetime - pd.to_datetime(df['采购入库日期'])).dt.days
df.head()

,sku_id,当前库存,采购入库日期,采购入库数量,days
0,1007271270102,1296,2021-02-22,1296,8
1,1008903694810,1388,2021-02-22,1388,8
2,1006671011405,1296,2021-02-22,1296,8
3,1008229644607,240,2021-02-22,240,8
4,1006379474960,180,2021-02-22,180,8


现在我们需要对每一个 SKU 计算他的库龄， 我们可以通过group-apply的方式

In [5]:
def calc_stockage(gdf):
    
    if gdf.iloc[0]['当前库存']==0:
        return 0
                                           
    gdf['c0'] = gdf['采购入库数量'].cumsum()  # 累计采购入库数量                                       
    gdf['c1'] = gdf['c0'].shift(1).fillna(0) # 累计到前一次的采购入库数量
    
    def calc_belongs_current_row(row):
        if (row['当前库存'] - row['c0']) > 0:
            return row['采购入库数量']
        else:
            bc = row['当前库存'] - row['c1']
            return bc if bc>0 else 0
   
    gdf['c2'] = gdf.apply(calc_belongs_current_row, axis=1) # 当前库存中属于这次采购的部分 
    return gdf.apply(lambda x: int(x['c2']/x['当前库存'] * x['days']), axis=1).sum()

df_stockage = df.groupby('sku_id').apply(calc_stockage).to_frame(name='stockage').reset_index()
df_stockage.head()


,sku_id,stockage
0,1000014008898,85
1,1000015356938,540
2,1000015695798,610
3,1000020598014,453
4,1000020677953,22


In [6]:
df[df.sku_id==1000014008898]

,sku_id,当前库存,采购入库日期,采购入库数量,days
49,1000014008898,170,2021-02-08,15,22
208,1000014008898,170,2021-01-22,55,39
912,1000014008898,170,2020-10-29,2000,124
1153,1000014008898,170,2020-09-18,300,165
1355,1000014008898,170,2020-08-21,380,193
1439,1000014008898,170,2020-07-31,200,214
1717,1000014008898,170,2020-05-21,150,285
1737,1000014008898,170,2020-05-18,450,288


In [7]:
df_stockage.shape

(1090, 2)

每个SKU的库龄已经算出来了，现在我们来看一下他们的分布情况

In [8]:
df_stockage['stockage_group'] = pd.cut(df_stockage['stockage'], (0, 30, 60, 90, 180, 365, 365*5), right=False)
df_stockage.head()

,sku_id,stockage,stockage_group
0,1000014008898,85,"[60, 90)"
1,1000015356938,540,"[365, 1825)"
2,1000015695798,610,"[365, 1825)"
3,1000020598014,453,"[365, 1825)"
4,1000020677953,22,"[0, 30)"


每个 SKU 的当前库存数

In [9]:
df_stock = df.groupby('sku_id')[['当前库存']].first().reset_index()
df_stock.head()

,sku_id,当前库存
0,1000014008898,170
1,1000015356938,409
2,1000015695798,111
3,1000020598014,407
4,1000020677953,472


我们来看一下库龄分布情况

In [10]:
df_stockage.merge(df_stock).groupby('stockage_group').agg({
    'sku_id': pd.Series.nunique,
    'stockage': ['std', 'mean', 'median', 'max', 'min'],
    '当前库存': 'sum'
})

sku_id    stockage                                  当前库存
               nunique         std        mean median   max  min     sum
stockage_group                                                          
[0, 30)             46    6.119937   20.543478     21    29    7   22746
[30, 60)           158    6.660796   39.506329     38    59   30   58949
[60, 90)            74    9.038002   75.283784     74    89   60   28867
[90, 180)          215   24.437695  125.004651    123   178   90   68854
[180, 365)         226   40.574263  304.836283    309   364  180   99461
[365, 1825)        371  199.385834  595.126685    504  1160  367  131362